In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost
from pandas.plotting import scatter_matrix
from xgboost import XGBRegressor #ВНЕЗАПНО нужны вероятности отнесения к классу, а не предсказания классов.
from sklearn.model_selection  import KFold
#from sklearn.model_selection  import GridSearchCV

plt.style.use('ggplot')
%matplotlib inline

train_data = pd.read_csv('input/ml5/train.csv', sep = ";")
test_data = pd.read_csv('input/ml5/test.csv', sep = ";")



In [ ]:
print(train_data.shape)
train_data.head(10)

In [25]:
print(test_data.shape)
test_data.head()

(30000, 12)


,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
0,5,18888,1,154,85.0,130,80,1,1,0,0,1
1,6,19042,2,170,69.0,130,90,1,1,None,0,1
2,7,20432,1,160,70.0,120,75,1,1,0,0,0
3,10,18133,2,185,94.0,130,90,1,1,None,0,1
4,11,16093,2,170,76.0,120,80,1,1,0,0,1


In [ ]:
train_data.describe()

In [ ]:
test_data.describe()

In [ ]:

scatter_matrix(train_data[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc']], alpha=0.05, figsize=(10, 10));

In [ ]:
scatter_matrix(train_data[['ap_hi', 'ap_lo']], alpha=0.05, figsize=(10, 10));

Судя по распределению, частенько в рост/вес уходит давление 120/80
Есть выбросы со значениями больше 1000 (например, ap_lo) - явно лишний нолик справа

In [ ]:
print(test_data[test_data["height"]==0])

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(test_data[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc']], alpha=0.05, figsize=(10, 10));

In [ ]:
train_data[['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc']]

Тут на самом деле будет очистка данных и вот это всё. Но пока просто на сырых поиграем в xgboost

http://xgboost.readthedocs.io/en/latest/python/python_api.html

https://rdrr.io/cran/xgboost/man/xgb.cv.html

https://jessesw.com/XG-Boost/


In [20]:
train_data["ap_hi"][train_data["ap_hi"]<=0] = -1* train_data["ap_hi"]
test_data["ap_hi"][test_data["ap_hi"]<=0] = -1* test_data["ap_hi"]

train_data["ap_lo"][train_data["ap_lo"]<=0] = -1* train_data["ap_lo"]
test_data["ap_lo"][test_data["ap_lo"]<=0] = -1* test_data["ap_lo"]


train_data["ap_hi"][train_data["ap_hi"]>10000] = 0.01* train_data["ap_hi"]
test_data["ap_hi"][test_data["ap_hi"]>10000] = 0.01* test_data["ap_hi"]

train_data["ap_lo"][train_data["ap_lo"]>10000] = 0.01* train_data["ap_lo"]
test_data["ap_lo"][test_data["ap_lo"]>10000] = 0.01* test_data["ap_lo"]


train_data["ap_hi"][train_data["ap_hi"]>=500] = 0.1* train_data["ap_hi"]
test_data["ap_hi"][test_data["ap_hi"]>=500] = 0.1* test_data["ap_hi"]

train_data["ap_lo"][train_data["ap_lo"]>=500] = 0.1* train_data["ap_lo"]
test_data["ap_lo"][test_data["ap_lo"]>=500] = 0.1* test_data["ap_lo"]


train_data["height"][train_data["height"]==120] = 164
test_data["height"][test_data["height"]==120] = 164


C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing 

In [53]:
test_data["smoke"][test_data["smoke"]=="None"] = 0.088129
test_data["alco"][test_data["alco"]=="None"] = 0.053771
test_data["active"][test_data["active"]=="None"] = 0.824667

test_data = test_data.apply(pd.to_numeric)

C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\agurevich\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separat

In [ ]:
XGBRegressor().get_params().keys()

In [54]:
train_data.drop('id', axis=1, inplace=True)
test_data.drop('id', axis=1, inplace=True)

y_data = train_data['cardio']
train_data.drop('cardio', axis=1, inplace=True)

In [28]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

train_data = scaler.fit_transform(train_data)
test_data = scaler.transform(test_data)

In [55]:


kfold = KFold(n_splits=5, shuffle=True, random_state=42)
xgb_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
              'objective': 'binary:logistic', 'max_depth':5, 'min_child_weight':5, 'gamma':0.3,'reg_alpha':1.5}
 
    
xgdmat = xgboost.DMatrix(train_data, y_data)

#xgboost.cv(xgb_params, xgdmat, num_boost_round=500, metrics='logloss')

cv_xgb = xgboost.cv(params = xgb_params, dtrain = xgdmat, num_boost_round = 3000, nfold = 3,
                metrics = ['logloss'], # Make sure you enter metrics inside a list or you may encounter issues!
                early_stopping_rounds = 300) # Look for early stopping that minimizes error
cv_xgb.tail()
#0.539398

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
56,0.539277,0.000782,0.528856,0.000740
57,0.539284,0.000783,0.528735,0.000749
58,0.539284,0.000781,0.528576,0.000687
59,0.539275,0.000850,0.528420,0.000677
60,0.539260,0.000826,0.528265,0.000691


In [42]:
from sklearn.grid_search import GridSearchCV 
from xgboost.sklearn import XGBClassifier
param_test1 = {
 'reg_alpha':[1,1.5, 2]
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, min_child_weight=5, max_depth=5, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic'), 
 param_grid = param_test1, scoring='neg_log_loss',iid=False, cv=5)
gsearch1.fit(train_data,y_data)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: -0.53862, std: 0.00413, params: {'reg_alpha': 1},
  mean: -0.53859, std: 0.00409, params: {'reg_alpha': 1.5},
  mean: -0.53860, std: 0.00410, params: {'reg_alpha': 2}],
 {'reg_alpha': 1.5},
 -0.5385945263250188)

In [56]:
final_gb = xgboost.train(xgb_params, xgdmat, num_boost_round = 60)

In [57]:
testdmat = xgboost.DMatrix(test_data)
y_pred = final_gb.predict(testdmat) # Predict using our testdmat
y_pred

array([ 0.46853283,  0.57263577,  0.41307732, ...,  0.45517501,
        0.34185824,  0.71280122], dtype=float32)

In [58]:
np.savetxt("ml5_predicto.csv", y_pred, delimiter=",")

In [40]:
y_pred.shape

(30000,)